In [1]:
import glob
import os
import json
import librosa
import numpy as np
import torch
import argparse
from muq import MuQ
from hydra.utils import instantiate
from omegaconf import OmegaConf
from safetensors.torch import load_file
from tqdm import tqdm

d:\Anaconda\envs\songeval\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class Synthesizer(object):

    def __init__(self,
                 checkpoint_path,
                 input_path,
                 output_dir,
                 use_cpu: bool = False):
    
        self.checkpoint_path = checkpoint_path
        self.input_path = input_path
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)
        self.device = torch.device('cuda') if (torch.cuda.is_available() and (not use_cpu)) else torch.device('cpu')
    
    @torch.no_grad()
    def setup(self):
        
        train_config = OmegaConf.load(os.path.join(os.path.dirname(self.checkpoint_path), '../config.yaml'))
        print(train_config)
        model = instantiate(train_config.generator).to(self.device).eval()
        state_dict = load_file(self.checkpoint_path, device="cpu")
        model.load_state_dict(state_dict, strict=False)

        self.model = model
        self.muq = MuQ.from_pretrained("OpenMuQ/MuQ-large-msd-iter")
        self.muq = self.muq.to(self.device).eval()
        self.result_dcit = {}
        
    @torch.no_grad()
    def synthesis(self):
        if os.path.isfile(self.input_path):
            if self.input_path.endswith(('.wav', '.mp3')):
                lines = []
                lines.append(self.input_path)
            else:
                with open(self.input_path, "r") as f:
                    lines = [line for line in f]
            input_files = [{
                "input_path": line.strip(), 
            } for line in lines]
            print(f"input filelst: {self.input_path}")
        elif os.path.isdir(self.input_path):
            input_files = [{
                "input_path": file,
            }for file in glob.glob(os.path.join(self.input_path, '*')) if file.lower().endswith(('.wav', '.mp3'))]
        else:
            raise ValueError(f"input_path {self.input_path} is not a file or directory")
        
        
        for input in tqdm(input_files):
            self.handle(**input)
        with open(os.path.join(self.output_dir, "result.json") , "w")as f:
            json.dump(self.result_dcit, f, indent=4, ensure_ascii=False)
        
    @torch.no_grad()
    def handle(self, input_path):
        
        fid = os.path.basename(input_path).split('.')[0]
        if input_path.endswith('.npy'):
            input = np.load(input_path)
            
            # check ssl
            if len(input.shape) == 3 and input.shape[0] != 1:
                print('ssl_shape error', input_path)
                return
            if np.isnan(input).any():
                print('ssl nan', input_path)
                return
            
            input = torch.from_numpy(input).to(self.device)
            if len(input.shape) == 2:
                input = input.unsqueeze(0)

        if input_path.endswith(('.wav', '.mp3')):
            wav, sr = librosa.load(input_path, sr=24000)
            audio = torch.tensor(wav).unsqueeze(0).to(self.device)
            output = self.muq(audio, output_hidden_states=True)
            input = output["hidden_states"][6]
            
        values = {} 
        scores_g = self.model(input).squeeze(0)
        values['Coherence'] = round(scores_g[0].item(), 4)
        values['Musicality'] = round(scores_g[1].item(), 4)
        values['Memorability'] = round(scores_g[2].item(), 4)
        values['Clarity'] = round(scores_g[3].item(), 4)
        values['Naturalness'] = round(scores_g[4].item(), 4)
        self.result_dcit[fid] = values
        

In [12]:
ckpt_path = "ckpt/model.safetensors"
input_path = './path/to/snow.mp3'  # replace with your input path
output_dir = './path/to/output'  # replace with your output path
assert os.path.exists(ckpt_path), f"checkpoint {ckpt_path} not exists!"
assert os.path.exists(input_path), f"input path {input_path} not exists!"
assert os.path.isdir(output_dir), f"output dir {output_dir} not exists!"
synthesizer = Synthesizer(checkpoint_path=ckpt_path,
                              input_path=input_path,
                              output_dir=output_dir,
                              use_cpu=False)


In [13]:
synthesizer.setup()

{'generator': {'_target_': 'model.Generator', 'in_features': 1024, 'ffd_hidden_size': 4096, 'num_classes': 5, 'attn_layer_num': 4}}


d:\Anaconda\envs\songeval\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [14]:
# handle
synthesizer.synthesis()

input filelst: ./path/to/snow.mp3


100%|██████████| 1/1 [00:43<00:00, 43.22s/it]


In [15]:
torch.cuda.is_available()

False

In [16]:
print(torch.__version__)

2.7.0+cpu
